# 14 构建流程和软件

自动化构建流水线和软件的步骤：
1．配置。检测平台相关的变量和用户指定的选项，使用内容设置后续步骤。
2．编译。依靠编译器将源代码转换为机器可读的二进制格式。只在使用编译型语言编写的软件时才需要。
3．链接。为二进制格式的库或可执行文件添加依赖库。
4．安装。在Python环境中进行的。即将编译的库、可执行文件、数据或其他文件放置在文件系统中可访问的位置。
5．运行。让用户实际运行分析流程或软件。

## 14.1 make

make：一个命令行实用程序，用于确定处理流程中需要执行哪些元素。
用来自动化处理所有具有依赖关系的流程，即各构建文件之间含有依赖关系。

#### 例：更新论文

首先需要将新数据添加到分析工具的数据文件列表中，接着使用所有新旧数据重新运行分析程序。
重新运行生成图像的绘图程序。
使用LaTeX重新编译论文。

需要处理由许多文件组成的依赖关系树，必须重新生成依赖该文件的文件。

make工具能够有效地自动化进行步骤。
能跟踪文件彼此的依赖关系，并检测哪些部分需要更新。

### 14.1.1 运行make

运行语法：<code>make [ -f makefile ] [ options ] ... [ targets ] ...</code>

无参数的make运行需要makefile。

### 14.1.2 makefile

makefile：一个遵循特定格式规则的纯文本文件，为make描述文件和任务之间的完整依赖关系树。
还描述了更新文件的步骤（即更新树中的节点需要执行哪些命令）。

如果没有为make命令提供参数，那么make会尝试在当前目录中寻找一个名为Makefile的文件。

### 14.1.3 目标

目标：依赖关系树中的节点，通常是需要更新的文件。
makefile主要用“目标—先决条件—行为”这样的语法来定义目标：

```text
target : prerequisites  # 冒号将目标名称与先决条件列表隔开
         action  # action前必须有一个缩进
```

target是.dat文件。
分析中的.dat文件由raw_data目录中的原始.h5文件生成，使用bash脚本实现。
按以上更新：

```text
# 要创建或更新的目标文件是photon_photon.data文件。依赖文件是shell脚本和.h5文件
photon_photon.dat : photon_analysis.sh ./raw_data/ *.h5
# 根据先决条件更新目标时必须执行这条命令
         ./photon_analysis.sh -n=2 > photon_photon.dat
```

如果执行make photon_photon.dat，那么make将进行下列操作：
1．检查先决条件和photon_photon.dat的状态。
2．如果时间戳发生变化，将执行行为。
3．如果时间戳没有改动，什么也不会发生，因为一切都是最新的。

makefile由许多这样的“目标—先决条件—行为”的映射组成。因此，多个这样的指令能够构建完整的依赖关系树。

图片的更新：

```text
# 定义新目标fig4.svg
# fig4.svg文件依赖于photon photon.data作为先决条件
fig4.svg : photon_photon.dat plot_response.py
# 使用特定选项执行Python脚本来构建fig4.svg
        python plot_dat.py --input=photon_photon.dat --output=fig4.svg
```

### 14.1.4 特殊目标

没有参数默认情况下Makefile通常会运行第一个目标。
默认的行为是更新所有内容，习惯上将这个目标命名为“all”。
all目标可以使用通配符（*）定义。

## 14.2 构建和运行软件

从源代码安装的步骤：
.configure [options]  # 基于用户选项和系统特性创建makefile。
make [options]  # 构建步骤将源代码编译成二进制格式，并且添加依赖库的文件路径链接
make test  # 安装之前，最好执行test目标（如果有的话），以确保库在平台上成功构建
[sudo] make install  # 构建文件复制到计算机上的适当位置，通常是用户在配置步骤中指定的位置

这个过程要对应相应的系统和环境来进行。因此需要配置makefile。

### 14.2.1 Makefile配置

#### 自动化工具

自动化生成工具用于配置，功能包括：
1.检测平台和架构特性。
2.检测环境变量。
3.检测可用的命令、编译器和库。
4.接受用户输入。
5.生成定制的makefile。

配置工具处理了项目在构建阶段所有不确定的地方。
让开发人员为每个参数提供可行的默认值，在需要时可以用方法覆盖这些默认值。

自动化系统构建工具包括：CMake、Autotools (Automake + Autoconf)、SCons

#### 平台配置

大多数软件必须针对每个平台单独构建，但即使是最简单的事情也会因平台的不同而产生差异。
平台包括：Linux、UNIX、Windows、移动端、嵌入式

配置系统需要进行的工作：
1.确定系统平台
2.检测数据组织形式（系统位数等）

#### 系统和用户配置

构建系统基于不同用户的配置做出的调整：
1.使用什么编译器。
2.安装的库版本。
3.库的位置。
4.PATH和类似的环境变量有哪些目录（关键）。
5.构建的项目中有哪些可选部分。
6.使用什么编译器标志（调试构建、优化构建等）。

#### 依赖配置

一个软件必须利用另一个软件的功能才能运行，那么第二个软件称为依赖。

构建失败的可能原因：
1.构建系统找不到依赖库。
2.可用库不是正确的版本。

构建系统会在PATH、LD_LIBRARY_PATH和类似的环境变量中查找列出的库。
需要确保已经正确安装了所有依赖库。

### 14.2.2 编译

软件构建的makefile中的命令主要是编译器命令。

make命令能生成编译文件，并由makefile保存在临时目录中。
相当于实际安装之前的一个测试。
构建系统可能生成一个带有默认目标的makefile，该目标旨在将所有源代码编译到本地目录中。
在编译完成后，通常可以使用make test来测试构建结果。

## 14.3 安装

不同系统上的意义：
Windows：创建一个Setup.exe文件。
Python：实现一个setup.py或其他发布工具。
UNIX系统上的源代码：生成带有安装目标的makefile，以便调用make install。

升级/卸载程序：
Windows：由“添加/删除程序”工具处理。
UNIX：由包管理器处理。

安装程序需要拥有特定的平台相关功能，而这个功能通常由构建系统来处理。

在Linux上，创建软件包时不使用make install，而是用
<code>make DESTDIR = <a_fake_root_dir> install</code>
将软件包安装到假的根目录。然后从假根目录创建一个包。
